In [ ]:
import psutil
import requests
import time
import socket
from datetime import datetime

# Server endpoint where data will be sent
SERVER_URL = "http://192.168.29.9:5000/ingest"

# Your unique agent ID (can be hostname or UUID)
AGENT_ID =  socket.gethostname()

# The auth token expected by the server for authorization
AUTH_TOKEN = "psg_xdr_agent_token"

def get_system_metrics():
    mem = psutil.virtual_memory()
    disk = psutil.disk_usage('/')
    net = psutil.net_io_counters()

    return {
        "hostname": AGENT_ID,
        "timestamp": time.time(),
        "memory_total": mem.total,
        "memory_used": mem.used,
        "memory_available": mem.available,
        "memory_percent": mem.percent,
        "cpu_percent": psutil.cpu_percent(interval=1),
        "disk_total": disk.total,
        "disk_used": disk.used,
        "disk_free": disk.free,
        "disk_percent": disk.percent,
        "network_bytes_sent": net.bytes_sent,
        "network_bytes_recv": net.bytes_recv,
    }

def get_process_info():
    process_list = []

    for proc in psutil.process_iter(['pid', 'name', 'username', 'memory_percent', 'cmdline']):
        try:
            # Let CPU percent be calculated after small interval
            cpu = proc.cpu_percent(interval=0.1)
            open_files = proc.open_files()
            open_file_paths = [f.path for f in open_files] if open_files else []

            suspicious_extensions = ['.exe', '.dll', '.bat', '.cmd', '.scr']
            is_suspicious = any(
                file_path.lower().endswith(tuple(suspicious_extensions)) for file_path in open_file_paths
            )

            process_list.append({
                "agent_id": AGENT_ID,
                "timestamp": time.time(),
                "pid": proc.pid,
                "process_name": proc.name(),
                "username": proc.username(),
                "cpu_percent": cpu,
                "memory_percent": proc.memory_percent(),
                "cmdline": " ".join(proc.cmdline()) if proc.cmdline() else "",
                "open_file": ", ".join(open_file_paths),
                "is_suspicious": is_suspicious
            })

        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue

    return process_list

def send_metrics_to_server():
    headers = {
        "Authorization": f"Bearer {AUTH_TOKEN}"
    }

    system_metrics = get_system_metrics()
    system_metrics["processes"] = get_process_info()

    try:
        response = requests.post(SERVER_URL, json=system_metrics, headers=headers, timeout=10)
        if response.status_code == 200:
            print(f" Metrics sent successfully for agent {AGENT_ID}")
        else:
            print(f" Server returned status {response.status_code}: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f" Network or server error: {e}")

if __name__ == "__main__":
    while True:
        send_metrics_to_server()
        time.sleep(60)
# This script collects system and process metrics and sends them to a specified server endpoint.


 Metrics sent successfully for agent LAPTOP-LGQ3OFRV
 Metrics sent successfully for agent LAPTOP-LGQ3OFRV
 Metrics sent successfully for agent LAPTOP-LGQ3OFRV
 Metrics sent successfully for agent LAPTOP-LGQ3OFRV
 Server returned status 500: {
  "message": "1406 (22001): Data too long for column 'open_file' at row 1",
  "status": "error"
}

 Metrics sent successfully for agent LAPTOP-LGQ3OFRV


KeyboardInterrupt: 